<a href="https://colab.research.google.com/github/jyun0618/TAVE_MLab/blob/main/Chapter07/jiyun_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 7 앙상블 학습과 랜덤 포레스트

### #1
> 정확히 같은 훈련 데이터로 다섯 개의 다른 모델을 훈련시켜서 모두 95% 정확도를 얻었다면 이 모델들을 연결하여 더 좋은 결과를 얻을 수 있을까요? 가능하다면 어떻게 해야 할까요? 그렇지 않다면 왜일까요?

더 나은 결과를 기대할 수 있다. 모델이 서로 다르거나 다른 훈련 샘플에서 훈련되었다면 더욱 좋다.

### #2
> 직접 투표와 간접 투표 분류기의 차이점은 무엇일까요?

직접 투표: 앙상블에 있는 각 분류기의 선택을 카운트해서 가장 많은 투표를 얻은 클래스를 선택

간접 투표: 각 클래스의 평균적인 확률 추정값을 계산하여 가장 높은 확률을 가진 클래스를 선택

### #3
> 배깅 앙상블의 훈련을 여러 대의 서버에 분산시켜 속도를 높일 수 있을까요? 페이스팅 앙상블, 부스팅 앙상블, 랜덤 포레스트, 스태킹 앙상블의 경우는 어떨까요?

배깅 앙상블, 페이스팅 앙상블, 랜덤 포레스트: 각 예측기가 독립적이므로 여러 대의 서버에 분산하여 앙상블의 훈련 속도를 높일 수 있음

부스팅 앙상블: 이전 에측기를 기반으로 만들어지므로 훈련이 순차적이어야 함 → 분산해서 얻을 수 있는 이득 X

스태킹 앙상블: 한 층의 모든 예측기는 독립적 BUT 한 층에 있는 예측기들은 이전 층의 예측기들이 훈련된 후에 훈련될 수 있음

### #4
> OOB 평가의 장점은 무엇인가요?

배깅 앙상블의 각 예측기가 훈련에 포함되지 않은 샘플을 사용해 평가하기 때문에

추가적인 검증 세트가 없어도 편향되지 않게 앙상블을 평가하도록 도와줌

### #5
> 무엇이 엑스트라 트리 앙상블을 일반 랜덤 포레스트보다 더 랜덤하게 만드나요? 추가적인 무작위성이 어떻게 도움이 될까요? 엑스트라 트리 분류기는 일반 랜덤 포레스트보다 느릴까요, 빠를까요?

랜덤 포레스트처럼 각 노드에서 특성의 일부를 랜덤으로 선택해 분할에 사용

여기서 엑스트라 트리는 더 나아가 최선의 임계점을 찾는 것이 아닌 각 특성에 대해 랜덤한 임계점을 사용

또한 최선의 임계점을 찾지 않기 때문에 랜덤 포레스트보다 훨씬 빠르게 훈련

### #6
> AdaBoost 앙상블이 훈련 데이터에 과대적합되었다면 어떤 매개변수를 어떻게 바꾸어야 할까요?

훈련데이터에 과소적합되었다면 예측기의 수를 증가시키거나 기반 예측기의 하이퍼파라미터를 감소시켜보거나 학습률을 약간 증가시켜볼 수 있다.


### #7
> 그레이디언트 부스팅 앙상블이 훈련 데이터에 과대적합되었다면 학습률을 높여야 할까요, 낮춰야 할까요?

훈련세트에 과대적합되었다면 학습률을 감소시켜야 함. 예측기의 수가 너무 많으면 알맞은 개수를 찾기 위해 조기 종료 기법을 사용할 수 있음

### #8
> MNIST 데이터를 불러들여 훈련세트, 검증세트, 테스트세트로 나눕니다.

In [1]:
# from sklearn.datasets import fetch_mldata
# mnist = fetch_mldata('MNIST original')
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [5]:
from sklearn.model_selection import train_test_split

In [8]:
# train/val set과 test set 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
# train set과 val set 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

> 그런 다음 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기 같은 여러 종류의 분류기를 훈련시킵니다.

In [9]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [10]:
random_forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=10, random_state=42)
svm_clf = LinearSVC(max_iter=10000, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [11]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("훈련 예측기: ", estimator)
    estimator.fit(X_train, y_train)

훈련 예측기:  RandomForestClassifier(n_estimators=10, random_state=42)
훈련 예측기:  ExtraTreesClassifier(n_estimators=10, random_state=42)
훈련 예측기:  LinearSVC(max_iter=10000, random_state=42)
훈련 예측기:  MLPClassifier(random_state=42)


In [12]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9469, 0.9492, 0.911, 0.9614]

> 그리고 검증 세트에서 개별 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요.

* 직접 투표 분류기

In [13]:
from sklearn.ensemble import VotingClassifier

In [14]:
# 개별 예측기 지정
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf)
]

In [15]:
voting_clf = VotingClassifier(named_estimators)

In [16]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=10000, random_state=42)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [17]:
voting_clf.score(X_val, y_val)

0.9631

In [18]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.0, 0.0, 0.0, 0.0]

In [19]:
# svm 제거
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [20]:
# 예측기 목록 업데이트
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(n_estimators=10, random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(n_estimators=10, random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [21]:
# 훈련된 예측기는 업데이트 되지 않음
voting_clf.estimators_

[RandomForestClassifier(n_estimators=10, random_state=42),
 ExtraTreesClassifier(n_estimators=10, random_state=42),
 LinearSVC(max_iter=10000, random_state=42),
 MLPClassifier(random_state=42)]

In [22]:
# 훈련된 예측기에서 svm 제거
del voting_clf.estimators_[2]

In [23]:
# 재평가
voting_clf.score(X_val, y_val)

0.9648

* 간접 투표 분류기

In [24]:
voting_clf.voting = 'soft'

In [25]:
voting_clf.score(X_val, y_val)

0.97

>  앙상블을 얻고 나면 테스트 세트로 확인해보세요. 개별 분류기와 비교해서 성능이 얼마나 향상되나요?

In [26]:
voting_clf.score(X_test, y_test)

0.9683

In [28]:
[estimator.score(X_test, y_test) for estimator in estimators]

[0.9437, 0.9474, 0.9123, 0.9607]

### #9
> 이전 연습문제의 각 분류기를 실행해서 검증 세트에서 예측을 만들고 그 결과로 새로운 훈련 세트를 만들어보세요. 각 훈련 샘플은 하나의 이미지에 대한 전체 분류기의 예측을 담은 벡터이고 타깃은 이미지의 클래스입니다. 새로운 훈련 세트에 분류기 하나를 훈련시켜보세요.

In [33]:
import numpy as np
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [34]:
X_val_predictions

array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 2., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [35]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [36]:
rnd_forest_blender.oob_score_

0.9644

> 축하합니다. 방금 블렌더를 훈련시켰습니다. 그리고 이 분류기를 모아서 스태킹 앙상블을 구성했습니다! 이제 테스트 세트에 앙상블을 평가해보세요. 테스트 세트의 각 이미지에 대해 모든 분류기로 예측을 만들고 앙상블의 예측 결과를 만들기 위해 블렌더에 그 예측을 주입합니다. 앞서 만든 투표 분류기와 비교하면 어떤가요? 이제 StackingClassifier를 사용하여 다시 시도해보세요. 성능이 더 좋아졌나요? 그렇다면 그 이유는 무엇인가요?

In [37]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [38]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.963